In [1]:
import spotipy
import json
import webbrowser
import urllib.request
import spotipy.util as util
import pygeohash as pgh

In [2]:
with open("geocode_keys.txt", "r") as geokey_file:
    geo_key = geokey_file.read()

In [3]:
address1 = "15 Elm St, Totowa, NJ, 07512, US"
address2 = "39 E George St, Providence, RI 02906, US"

addresses = [address1, address2]
new_addresses = [address
                    .replace(",", "")
                    .replace(" ", "+") for address in addresses]

# print(new_addresses)

lat_list = []
lon_list = []
hash_list = []

for new_address in new_addresses:
    map_url = f"https://geocode.maps.co/search?q={new_address}&api_key={geo_key}"

    map_request = urllib.request.Request(map_url)
    map_response = urllib.request.urlopen(map_request)

    map_data = json.loads(map_response.read())
    lat = map_data[0]['lat']
    lon = map_data[0]['lon']

    # lat_list.append(lat)
    # lon_list.append(lon)

    hash_list.append(pgh.encode(float(lat), float(lon), precision=5))

print(hash_list)


['dr70z', 'drmjz']


In [4]:
# test_hash = pgh.encode(float(lat), float(lon), precision=5)
# print(test_hash)

In [5]:
with open("ticketmaster_keys.txt", "r") as key_file:
    ticketmaster_key = key_file.read()

with open("genreIDs.json", "r") as genreID_file:
    genre_list = json.loads(genreID_file.read())

# print(genre_list['rap'])

In [6]:
geohash = "gcpuv"
unit = "km"
radius = 10
genre = genre_list['country']

events_both_places = {}

for hash in hash_list:
    url = f"https://app.ticketmaster.com/discovery/v2/events.json?size=20&radius={radius}&genreId={genre}&geoPoint={hash}&classificationName=music&apikey={ticketmaster_key}"

    tm_request = urllib.request.Request(url)
    tm_response = urllib.request.urlopen(tm_request)

events_both_places = json.loads(tm_response.read())
event_names = events_both_places['_embedded']['events']

print(event_names)


[{'name': 'KANE BROWN: THE HIGH ROAD TOUR', 'type': 'event', 'id': 'vv1A8v-oyGA5mvai', 'test': False, 'url': 'https://www.ticketmaster.com/kane-brown-the-high-road-tour-providence-rhode-island-04-05-2025/event/010061479BB8126C', 'locale': 'en-us', 'images': [{'ratio': '3_2', 'url': 'https://s1.ticketm.net/dam/a/da0/08bf0082-1b91-4904-8004-401a80209da0_ARTIST_PAGE_3_2.jpg', 'width': 305, 'height': 203, 'fallback': False}, {'ratio': '16_9', 'url': 'https://s1.ticketm.net/dam/a/da0/08bf0082-1b91-4904-8004-401a80209da0_EVENT_DETAIL_PAGE_16_9.jpg', 'width': 205, 'height': 115, 'fallback': False}, {'ratio': '16_9', 'url': 'https://s1.ticketm.net/dam/a/da0/08bf0082-1b91-4904-8004-401a80209da0_SOURCE', 'width': 2427, 'height': 1365, 'fallback': False}, {'ratio': '4_3', 'url': 'https://s1.ticketm.net/dam/a/da0/08bf0082-1b91-4904-8004-401a80209da0_CUSTOM.jpg', 'width': 305, 'height': 225, 'fallback': False}, {'ratio': '3_2', 'url': 'https://s1.ticketm.net/dam/a/da0/08bf0082-1b91-4904-8004-401a80

In [ ]:
# if len(event_names) < 5:
#     radius*2
#     full_event_listings = json.loads(tm_response.read())


artist_names = [artist['name'] for artist in event_names]
artist_names_no_repeats = list(dict.fromkeys(artist_names))
print(artist_names_no_repeats)

['KANE BROWN: THE HIGH ROAD TOUR', 'Lucinda Williams', 'Jason Isbell']


In [8]:
cred = "spotify_keys.json"
with open(cred, "r") as spotify_key_file:
    api_tokens = json.load(spotify_key_file)

spotify_key_file.close()

client_id = api_tokens['client_id']
client_secret = api_tokens['client_secret']
redirectURI = api_tokens['redirect']
username = api_tokens['username']

scope = 'user-read-private user-read-playback-state user-modify-playback-state playlist-modify-public user-library-read'
token = util.prompt_for_user_token(username, scope, client_id=client_id,
                           client_secret=client_secret,
                           redirect_uri=redirectURI)

In [16]:
sp = spotipy.Spotify(auth=token)

playlist_songs = []

for artist in artist_names_no_repeats:
    returned_song = sp.search(q=artist, type="track", limit=1)

    if returned_song['tracks']['items'][0]['uri'] not in playlist_songs:
        playlist_songs.append(returned_song['tracks']['items'][0]['uri'])

# print(playlist_songs)

empty_playlist = sp.user_playlist_create(user=username, name=f"Journey from {address1.upper()} to {address2.upper()}", public=True, description=f"Artists who've played recently anywhere between {address1} and {address2}")
full_playlist = sp.user_playlist_add_tracks(username, empty_playlist['id'], playlist_songs)

In [15]:
webbrowser.open(empty_playlist['external_urls']['spotify'])

True